##[Ссылка на датасет](https://huggingface.co/datasets/Rianknow/creditscoring)


In [4]:
!pip install category_encoders
!pip install catboost
!pip install optuna
!pip install lime -q
!pip install shap -q
!pip install eli5 -q
!pip install phik -q
!pip install catboost
!pip install optuna
!pip install optuna-integration[catboost]
!pip install optuna catboost -q
import optuna
from optuna.integration import CatBoostPruningCallback
import phik
from phik.report import plot_correlation_matrix
from phik import report
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import pickle
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.preprocessing import PolynomialFeatures
from tqdm import tqdm
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import shap
import lime
from catboost import CatBoostRegressor, Pool, CatBoostClassifier


# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier
# from xgboost import XGBClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score
# import numpy as np
# import pandas as pd
# import pickle
# import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
# %matplotlib inline
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# from sklearn import tree
# from sklearn.preprocessing import PolynomialFeatures
# from tqdm import tqdm
# from sklearn.preprocessing import OrdinalEncoder
# from sklearn.model_selection import KFold, StratifiedKFold
# from sklearn.model_selection import train_test_split
# from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.preprocessing import LabelEncoder
# import shap
# import lime


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
z

In [5]:
df = pd.read_csv('./dataset/creditscoring/train.csv', sep = ',')
df.replace('_', np.nan, inplace = True)
df.head()

/var/folders/_m/n57jfftj32jbkm7dyvz9gpj80000gn/T/ipykernel_78143/3777082345.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./dataset/creditscoring/train.csv', sep = ',')


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,NaN,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [6]:
# unique_counts = df['target'].value_counts()

# Вывод результатов
# print(unique_counts)

In [7]:
df.dtypes

ID                           object
Customer_ID                  object
Month                        object
Name                         object
Age                          object
SSN                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Type_of_Loan                 object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
Credit_Score                

In [8]:
df.shape

# df.drop(columns='Name', inplace=True, axis=1) # Удаление столбца

(100000, 28)

In [9]:
print((df.isnull().sum())/df.shape[0]*100)


ID                           0.000
Customer_ID                  0.000
Month                        0.000
Name                         9.985
Age                          0.000
SSN                          0.000
Occupation                   0.000
Annual_Income                0.000
Monthly_Inhand_Salary       15.002
Num_Bank_Accounts            0.000
Num_Credit_Card              0.000
Interest_Rate                0.000
Num_of_Loan                  0.000
Type_of_Loan                11.408
Delay_from_due_date          0.000
Num_of_Delayed_Payment       7.002
Changed_Credit_Limit         2.091
Num_Credit_Inquiries         1.965
Credit_Mix                  20.195
Outstanding_Debt             0.000
Credit_Utilization_Ratio     0.000
Credit_History_Age           9.030
Payment_of_Min_Amount        0.000
Total_EMI_per_month          0.000
Amount_invested_monthly      4.479
Payment_Behaviour            0.000
Monthly_Balance              1.200
Credit_Score                 0.000
dtype: float64


In [10]:
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].str.replace('_', '')
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].str.replace('__', '')
df['Monthly_Balance'] = df['Monthly_Balance'].str.replace('__', '')
df['Age'] = df['Age'].str.replace('_', '')
df['Annual_Income'] = df['Annual_Income'].str.replace('_', '')
df['Outstanding_Debt'] = df['Outstanding_Debt'].str.replace('_', '')
df['Num_of_Loan'] = df['Num_of_Loan'].str.replace('_', '')
df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].str.replace('_', '')

df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].astype(float)
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].astype(float)
df['Monthly_Balance'] = df['Monthly_Balance'].astype(float)
df['Age'] = df['Age'].astype(int)
df['Annual_Income'] = df['Annual_Income'].astype(float)
df['Outstanding_Debt'] = df['Outstanding_Debt'].astype(float)
df['Num_of_Loan'] = df['Num_of_Loan'].astype(float)
df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].astype(float)

In [11]:
# dropCol = ['ID', 'SSN', 'Name', 'Customer_ID', 'Credit_History_Age']

dropCol = [
  'ID', 
  'SSN', 
  'Name', 
  'Customer_ID', 
  'Credit_History_Age', 
  'Occupation', 
  'Num_Bank_Accounts', 
  'Num_Credit_Card', 
  # 'Type_of_Loan', 
  'Interest_Rate', 
  'Num_of_Delayed_Payment', 
  'Changed_Credit_Limit', 
  'Credit_Mix', 
  'Outstanding_Debt', 
  'Credit_Utilization_Ratio', 
  'Payment_of_Min_Amount', 
  'Amount_invested_monthly',
  'Monthly_Balance',
  'Total_EMI_per_month', 
  'Payment_Behaviour',
  'Delay_from_due_date',
]


for col in dropCol:
  df.drop(columns=col, inplace=True, axis=1)

cat_features = ['Type_of_Loan']

for col_name, col_type in df.dtypes.items():
  if (col_type == object):
    cat_features.append(col_name)

del cat_features[cat_features.index('Credit_Score')]

cat_features


['Type_of_Loan', 'Month', 'Type_of_Loan']

In [12]:
df.dtypes

Month                     object
Age                        int64
Annual_Income            float64
Monthly_Inhand_Salary    float64
Num_of_Loan              float64
Type_of_Loan              object
Num_Credit_Inquiries     float64
Credit_Score              object
dtype: object

In [13]:
def for_cat_Nan(df, features):
    for feature in features:
        mode_value = df[feature].mode()[0]
        df[feature].fillna(mode_value, inplace=True)
def for_num_Nan(df, features):
    for feature in features:
        median_value = df[feature].mode()[0]
        df[feature].fillna(median_value, inplace=True)

# cat_Nan = ['Type_of_Loan', 'Credit_Mix']
# num_Nan = ['Monthly_Inhand_Salary', 'Num_Credit_Inquiries', 'Amount_invested_monthly', 'Num_of_Delayed_Payment', 'Monthly_Balance', 'Changed_Credit_Limit']
cat_Nan = ['Type_of_Loan']
num_Nan = []

for_cat_Nan(df,cat_Nan)
for_num_Nan(df,num_Nan)

/var/folders/_m/n57jfftj32jbkm7dyvz9gpj80000gn/T/ipykernel_78143/336795824.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[feature].fillna(mode_value, inplace=True)


In [14]:
print((df.isnull().sum())/df.shape[0]*100)

Month                     0.000
Age                       0.000
Annual_Income             0.000
Monthly_Inhand_Salary    15.002
Num_of_Loan               0.000
Type_of_Loan              0.000
Num_Credit_Inquiries      1.965
Credit_Score              0.000
dtype: float64


In [15]:
df.dtypes

Month                     object
Age                        int64
Annual_Income            float64
Monthly_Inhand_Salary    float64
Num_of_Loan              float64
Type_of_Loan              object
Num_Credit_Inquiries     float64
Credit_Score              object
dtype: object

In [16]:
label_encoder = LabelEncoder()

for col_name, col_type in df.dtypes.items():
  if (col_type == object):
    print(df[col_name].value_counts(), end='\n\n')
    df[col_name] = label_encoder.fit_transform(df[col_name])

Month
January     12500
February    12500
March       12500
April       12500
May         12500
June        12500
July        12500
August      12500
Name: count, dtype: int64

Type_of_Loan
Not Specified                                                                                                                         12816
Credit-Builder Loan                                                                                                                    1280
Personal Loan                                                                                                                          1272
Debt Consolidation Loan                                                                                                                1264
Student Loan                                                                                                                           1240
                                                                                                              

In [17]:
df.rename(columns={'Credit_Score': 'target'}, inplace=True)

In [18]:
df.head()

,Month,Age,Annual_Income,Monthly_Inhand_Salary,Num_of_Loan,Type_of_Loan,Num_Credit_Inquiries,target
0,3,23,19114.12,1824.843333,4.0,128,4.0,0
1,2,23,19114.12,NaN,4.0,128,4.0,0
2,6,-500,19114.12,NaN,4.0,128,4.0,0
3,0,23,19114.12,NaN,4.0,128,4.0,0
4,7,23,19114.12,1824.843333,4.0,128,4.0,0


In [19]:
df.dtypes

Month                      int64
Age                        int64
Annual_Income            float64
Monthly_Inhand_Salary    float64
Num_of_Loan              float64
Type_of_Loan               int64
Num_Credit_Inquiries     float64
target                     int64
dtype: object


---
## Валидация


In [20]:
# Разделение на три выборки (трэин / тест / валидация)
X = df.drop(columns='target')
y = df['target']
X_t, X_test, y_t, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    #shuffle=True,
                                                    stratify=y,
                                                    random_state=100)
X_train, X_val, y_train, y_val = train_test_split(X_t, y_t,
                                                    test_size=0.25,
                                                   # shuffle=True,
                                                    stratify=y_t,
                                                    random_state=100)

In [21]:
# with open('./saved_model/model.pkl', 'wb') as file:
#     pickle.dump(clf_for_catboost, file)

In [22]:
# Чистим колокнки
cat_features_copy = ['Month',
 'Occupation',
 'Type_of_Loan',
 'Credit_Mix',
 'Payment_of_Min_Amount',
 'Payment_Behaviour'];
targets = ["target"]
features2drop = []
filtered_features = [i for i in X_train.columns if (i not in targets and i not in features2drop)]
num_features = [i for i in filtered_features if i not in cat_features]

In [23]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score


# Обучение модели CatBoost с лучшими гиперпараметрами
clf = CatBoostClassifier(
    thread_count=-1,
    random_seed=100,
    cat_features=cat_features,
)

# Обучение на тренировочных данных
clf.fit(
    X_train,
    y_train,
    eval_set=(X_val, y_val),
    verbose=0,
    plot=False,
    early_stopping_rounds=50,
)

# Предсказание на тренировочном наборе
y_train_pred = clf.predict(X_train)

# Предсказание на валидационном наборе
y_val_pred = clf.predict(X_train)

# Предсказание на тестовом наборе
y_test_pred = clf.predict(X_test)

# Вычисление и вывод точности и F1-меры
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_f1 = f1_score(y_train, y_train_pred, average = 'weighted')
test_f1 = f1_score(y_test, y_test_pred, average = 'weighted')

print("Accuracy on train set: {:.4f}".format(train_accuracy))
print("Accuracy on test set: {:.4f}".format(test_accuracy))

print("F1 score on train set: {:.4f}".format(train_f1))
print("F1 score on test set: {:.4f}".format(test_f1))

Accuracy on train set: 0.7547
Accuracy on test set: 0.7359
F1 score on train set: 0.7483
F1 score on test set: 0.7295


In [24]:
df.dtypes

Month                      int64
Age                        int64
Annual_Income            float64
Monthly_Inhand_Salary    float64
Num_of_Loan              float64
Type_of_Loan               int64
Num_Credit_Inquiries     float64
target                     int64
dtype: object

In [25]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score


# Обучение модели CatBoost с лучшими гиперпараметрами
clf = CatBoostClassifier(
    thread_count=-1,
    random_seed=100,
    cat_features=cat_features,
)

# Обучение на тренировочных данных
clf.fit(
    X_train,
    y_train,
    eval_set=(X_val, y_val),
    verbose=0,
    plot=False,
    early_stopping_rounds=50,
)

# Предсказание на тренировочном наборе
y_train_pred = clf.predict(X_train)

# Предсказание на валидационном наборе
y_val_pred = clf.predict(X_val)

# Предсказание на тестовом наборе
y_test_pred = clf.predict(X_test)

# Вычисление и вывод точности и F1-меры
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

train_f1 = f1_score(y_train, y_train_pred, average = 'weighted')
test_f1 = f1_score(y_test, y_test_pred, average = 'weighted')

print("Accuracy on train set: {:.4f}".format(train_accuracy))
print("Accuracy on test set: {:.4f}".format(test_accuracy))

print("F1 score on train set: {:.4f}".format(train_f1))
print("F1 score on test set: {:.4f}".format(test_f1))

Accuracy on train set: 0.7547
Accuracy on test set: 0.7359
F1 score on train set: 0.7483
F1 score on test set: 0.7295


In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier

# Предполагаем, что df_for_auto уже определен и содержит три класса в столбце 'target'
X = df.drop(['target'], axis=1)
y = df['target']
n_splits = 10

clfs = [] # Модели, которые обучаю
scores = [] # Оценка предсказаний точности
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=7575)

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Разделение обучающего набора на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=7575)

    # Создайте CatBoostClassifier для многоклассовой классификации
    clf_for_catboost = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.1,
        depth=6,
        cat_features=cat_features,
        eval_metric='MultiClass',  # Установите метрику для многоклассовой классификации
        random_seed=7575,
        verbose=0  # Установите в 0, чтобы отключить вывод во время обучения
    )

    clfs.append(clf_for_catboost)

    # Обучение модели на обучающей выборке
    clf_for_catboost.fit(X_train, y_train)

    # Прогнозирование на тестовом наборе
    y_pred = clf_for_catboost.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    scores.append(score)

    print(f"fold: acc: {score}")

assert len(clfs) == n_splits
print("mean accuracy score --", np.mean(scores), np.std(scores).round(4))

fold: acc: 0.7369
fold: acc: 0.7352
fold: acc: 0.7328
fold: acc: 0.7308
fold: acc: 0.726
fold: acc: 0.7298
fold: acc: 0.7305
fold: acc: 0.7282
fold: acc: 0.7417
fold: acc: 0.7321
mean accuracy score -- 0.7323999999999999 0.0043


In [27]:
import pickle
with open('./saved_model/model.pkl', 'wb') as file:
    pickle.dump(clf_for_catboost, file)

In [33]:
set(df['Type_of_Loan'])

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [2]:
import pickle
import pandas as pd

with open('./saved_model/model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

new_data = pd.DataFrame({
    'Month': [0],  # или другой месяц, который вам нужен
    'Age': [28],  # возраст клиента
    'Annual_Income': [75000.0],  # годовой доход
    'Monthly_Inhand_Salary': [5000.0],  # ежемесячная зарплата после налогов
    'Num_of_Loan': [1],  # количество текущих кредитов
    # 'Delay_from_due_date': [0],  # задержка по выплатам
    'Type_of_Loan': [3319],  # Тип кредита
    'Num_Credit_Inquiries': [2],  # количество запросов на кредит
    'Credit_Score': [0]  # кредитный рейтинг
})

# Month                      int64
# Age                        int64
# Annual_Income            float64
# Monthly_Inhand_Salary    float64
# Num_of_Loan              float64
# Type_of_Loan               int64
# Num_Credit_Inquiries     float64
# target                     int64

# Шаг 3: Предобработка данных (если требуется)
# Если ваша модель требует какого-то предварительного преобразования данных
# (например, кодирование категориальных переменных или масштабирование числовых),
# сделайте это здесь. Пример кодирования выборки:

# Пример кодирования категориальных переменных
# new_data_encoded = pd.get_dummies(new_data, columns=['Month', 'Credit_Score'])

# Убедитесь, что все необходимые столбцы совпадают с обученной моделью
# Если какие-то столбцы отсутствуют, добавьте их с нулевыми значениями
# for col in model.feature_names_in_:
#     if col not in new_data_encoded.columns:
#         new_data_encoded[col] = 0.0

# # Убедитесь, что столбцы совпадают по порядку и типу
# new_data_encoded = new_data_encoded[model.feature_names_in_]

# Шаг 4: Использование модели для предсказания
predictions = model.predict(new_data)

# Шаг 5: Вывод результата
print("Предсказания: ", predictions)

ModuleNotFoundError: No module named 'pandas'

In [50]:
classes = label_encoder.classes_
print("Закодированные метки:", df['Type_of_Loan'][265])
# print("Классы:", classes)

# mapping = {classes[i]: df['Type_of_Loan'][i] for i in range(len(classes))}
# print("Словарь соответствия:", mapping)

# encoded_labels = label_encoder.fit_transform(data)


Закодированные метки: 3319
